# Лабораторная работа №3: Проведение исследований с решающим деревом

## 1. Выбор начальных условий

Был проведен в ЛР №1.

In [3]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [4]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# Скачивание Date Fruit Dataset
date_fruit_path = kagglehub.dataset_download("muratkokludataset/date-fruit-datasets")

# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

100%|██████████| 408k/408k [00:00<00:00, 40.3MB/s]

Extracting files...


In [6]:
# Скачивание Concrete Compressive Strength
concrete_strength_path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")

# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

100%|██████████| 14.1k/14.1k [00:00<00:00, 12.9MB/s]

Extracting files...


## 2. Создание бейзлайна и оценка качества

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score, make_scorer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

Разделим датасет для классификации на обучающую и тестовую выборки

In [8]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [9]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [10]:
tree_classifier = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_classifier.fit(X_train_class, y_train_class)

y_pred_class = tree_classifier.predict(X_test_class)

accuracy_tree = accuracy_score(y_test_class, y_pred_class)
f1_tree = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Decision Tree Classifier - Accuracy: {accuracy_tree:.4f}, F1-Score: {f1_tree:.4f}")

Decision Tree Classifier - Accuracy: 0.8056, F1-Score: 0.7969


In [11]:
tree_regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = tree_regressor.predict(X_test_reg)

rmse_tree = root_mean_squared_error(y_test_reg, y_pred_reg)
r2_tree = r2_score(y_test_reg, y_pred_reg)

print(f"Decision Tree Regressor - RMSE: {rmse_tree:.4f}, R²: {r2_tree:.4f}")

Decision Tree Regressor - RMSE: 9.5792, R²: 0.6439


Итак, точность для встроенной в Sklearn модели решающего дерева получилась приемлемой (80.5% точности для классификатора, 9.57 RMSE для регрессора). Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Будем перебирать гиперпараметры решающего дерева, такие, как max_depth, min_samples_split, min_samples_leaf

In [12]:
# 1. Подбор гиперпараметров для DecisionTreeClassifier
print("=== Decision Tree Classifier - Hyperparameter Tuning ===")
param_grid_classifier = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search_classifier = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid_classifier,
    scoring=make_scorer(f1_score, average='weighted'),
    cv=5,
    verbose=1
)

grid_search_classifier.fit(X_train_class, y_train_class)

# Лучшие параметры и результат
best_params_classifier = grid_search_classifier.best_params_
best_score_classifier = grid_search_classifier.best_score_

print(f"Лучшие параметры для классификатора: {best_params_classifier}")
print(f"Лучший F1-Score на кросс-валидации: {best_score_classifier:.4f}")

# 2. Подбор гиперпараметров для DecisionTreeRegressor
print("\n=== Decision Tree Regressor - Hyperparameter Tuning ===")
param_grid_regressor = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search_regressor = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid_regressor,
    scoring='neg_root_mean_squared_error',
    cv=5,
    verbose=1
)

grid_search_regressor.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат
best_params_regressor = grid_search_regressor.best_params_
best_score_regressor = -grid_search_regressor.best_score_

print(f"Лучшие параметры для регрессора: {best_params_regressor}")
print(f"Лучший RMSE на кросс-валидации: {best_score_regressor:.4f}")

=== Decision Tree Classifier - Hyperparameter Tuning ===
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Лучшие параметры для классификатора: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Лучший F1-Score на кросс-валидации: 0.8272

=== Decision Tree Regressor - Hyperparameter Tuning ===
Fitting 5 folds for each of 36 candidates, totalling 180 fits
Лучшие параметры для регрессора: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Лучший RMSE на кросс-валидации: 7.1226


Обучим модели с лучшими гиперпараметрами

In [13]:
# 1. Обучение DecisionTreeClassifier с лучшими параметрами
print("\n=== Обучение Decision Tree Classifier с лучшими параметрами ===")
best_tree_classifier = DecisionTreeClassifier(
    max_depth=best_params_classifier['max_depth'],
    min_samples_split=best_params_classifier['min_samples_split'],
    min_samples_leaf=best_params_classifier['min_samples_leaf'],
    random_state=42
)

best_tree_classifier.fit(X_train_class, y_train_class)

# Предсказания на тестовой выборке
y_pred_class = best_tree_classifier.predict(X_test_class)

# Оценка качества
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# 2. Обучение DecisionTreeRegressor с лучшими параметрами
print("\n=== Обучение Decision Tree Regressor с лучшими параметрами ===")
best_tree_regressor = DecisionTreeRegressor(
    max_depth=best_params_regressor['max_depth'],
    min_samples_split=best_params_regressor['min_samples_split'],
    min_samples_leaf=best_params_regressor['min_samples_leaf'],
    random_state=42
)

best_tree_regressor.fit(X_train_reg, y_train_reg)

# Предсказания на тестовой выборке
y_pred_reg = best_tree_regressor.predict(X_test_reg)

# Оценка качества
rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")


=== Обучение Decision Tree Classifier с лучшими параметрами ===
Test Accuracy: 0.8556
Test F1-Score: 0.8590

=== Обучение Decision Tree Regressor с лучшими параметрами ===
Test RMSE: 6.6985
Test R²: 0.8259


Итак, точность обоих моделей возросла, причем для регрессора - существенно (RMSE 6.69 против 9.57)

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию решающего дерева для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn.

In [20]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature          # Признак для разбиения
        self.threshold = threshold      # Пороговое значение
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Значение в листе (для терминальных узлов)

class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _gini(self, y):
        proportions = np.bincount(y) / len(y)
        return 1 - np.sum(proportions ** 2)

    def _split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

    def _best_split(self, X, y):
        best_score = float('inf')  # Лучший критерий разбиения (минимальный)
        best_split = None

        for feature in range(X.shape[1]):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature])  # Уникальные значения признака
            for threshold in thresholds:
                # Разбиение данных
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                # Проверка, что оба подмножества достаточно велики
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                # Вычисление критерия Gini
                score = (len(y_left) * self._gini(y_left) + len(y_right) * self._gini(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        # Условие остановки
        if (len(np.unique(y)) == 1 or depth == self.max_depth or
            len(y) < self.min_samples_split):
            return DecisionTreeNode(value=np.bincount(y).argmax())

        # Найти лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        feature, threshold = best_split

        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.bincount(y).argmax())

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.root = None

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def _split(self, X, y, feature, threshold):
        left_indices = np.where(X[:, feature] < threshold)[0]
        right_indices = np.where(X[:, feature] >= threshold)[0]
        return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

    def _best_split(self, X, y):
        best_score = float('inf')  # Лучший критерий разбиения (минимальный MSE)
        best_split = None

        for feature in range(X.shape[1]):  # Перебор всех признаков
            thresholds = np.unique(X[:, feature])  # Уникальные значения признака
            for threshold in thresholds:
                # Разбиение данных
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)

                # Проверка, что оба подмножества достаточно велики
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                # Вычисление MSE
                score = (len(y_left) * self._mse(y_left) + len(y_right) * self._mse(y_right)) / len(y)
                if score < best_score:
                    best_score = score
                    best_split = (feature, threshold)

        return best_split

    def _build_tree(self, X, y, depth):
        # Условие остановки
        if (len(np.unique(y)) == 1 or depth == self.max_depth or
            len(y) < self.min_samples_split):
            return DecisionTreeNode(value=np.mean(y))

        # Найти лучшее разбиение
        best_split = self._best_split(X, y)
        if best_split is None:
            return DecisionTreeNode(value=np.mean(y))

        feature, threshold = best_split

        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return DecisionTreeNode(value=np.mean(y))

        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)

        return DecisionTreeNode(feature, threshold, left_child, right_child)

    def fit(self, X, y):
        self.root = self._build_tree(X, y, 0)

    def _predict(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict(x, node.left)
        else:
            return self._predict(x, node.right)

    def predict(self, X):
        return np.array([self._predict(x, self.root) for x in X])

Обучим модели и оценим их качество

In [21]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)

custom_tree_classifier = DecisionTreeClassifierCustom(max_depth=5, min_samples_split=2)
custom_tree_classifier.fit(X_train_class, y_train_class)

y_pred_class = custom_tree_classifier.predict(X_test_class)
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Custom Decision Tree Classifier - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")

Custom Decision Tree Classifier - Accuracy: 0.7889, F1-Score: 0.7776


In [22]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

custom_tree_regressor = DecisionTreeRegressorCustom(max_depth=5, min_samples_split=2)
custom_tree_regressor.fit(X_train_reg, np.array(y_train_reg))

y_pred_reg = custom_tree_regressor.predict(X_test_reg)
rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Custom Decision Tree Regressor - RMSE: {rmse:.4f}, R²: {r2:.4f}")

Custom Decision Tree Regressor - RMSE: 9.2196, R²: 0.6701


Точность реализованных вручную моделей в целом близка к библиотечным.